In [2]:
# HW2 Benjamin Case
class Heap:
    """sources consulted: https://www.geeksforgeeks.org/leftist-tree-leftist-heap/"""
    __slots__ = "_right", "_left", "_rank", "_key", "_othervalues"

    def __init__(self, key, othervalues):
        self._left = None
        self._right = None
        self._rank = 1
        self._key = key
        self._othervalues = othervalues
        
    def findmin(self):
        return self
    
    def meld(h1, h2):
        """ merges the heaps h1 and h2 and returns a """
        if h1 == None:
            return h2
        else:
            if h2 == None:
                return h1
            else: # h1 not None and h2 not None
                if h1._key > h2._key:
                    h1, h2 = h2, h1
                return Heap.mesh(h1,h2)

    
    def mesh(h1,h2):
        """ subroutine in meld """
        if h1._left == None:
            h1._left = h2
        else: # h1._right not None
            h1._right = Heap.meld(h1._right,h2)
            
            if h1._left == None:  # make None less than any h1._right, since Python 3 does not compaire None and int
                h1._left, h1._right = h1._right, h1._left
            elif h2._right is not None and h1._left._rank < h1._right._rank: # make None less than any h1._right 
                h1._left, h1._right = h1._right, h1._left

        h1._rank = h1._rank + 1
        return h1
        
    def insert(self, key, othervalues):
        hnew = Heap(key,othervalues)
        self = Heap.meld(self, hnew)
        
    def deletemin(self):
        self = Heap.meld(self._left,self._right)
        return self

    
    def printout(self):
        print("entering printout")
        print(self._key, self._othervalues)
        if self._left is not None:
            print("left")
            self._left.printout()
        else: 
            print("self._left = None")
        if self._right is not None:
            print("right")
            self._right.printout()
        else:
            print("self._right = None")

In [3]:
class Partition:
    """Union-find structure for maintaining disjoint sets."""
    """code from the textbook: https://doc.lagout.org/programmation/python/Data%20Structures%20and%20Algorithms%20in%20Python%20[Goodrich,%20Tamassia%20&%20Goldwasser%202013-03-18].pdf"""

    #------------------------- nested Position class -------------------------
    class Position:
        __slots__ = '_container' , '_element' , '_size' , '_parent'
        
        def __init__(self, container, e):
            """Create a new position that is the leader of its own group."""
            self._container = container # reference to Partition instance
            self._element = e
            self._size = 1
            self._parent = self # convention for a group leader

        def element(self):
            """Return element stored at this position."""
            return self._element

    #------------------------- public Partition methods -------------------------
    def make_group(self, e):
        """Makes a new group containing element e, and returns its Position"""
        return self.Position(self, e)
    
    def find(self, p):
        """Finds the group containging p and return the position of its leader."""
        if p._parent != p:
            p._parent = self.find(p._parent) # overwrite p. parent after recursion
        return p._parent
    
    def union(self, p, q):
        """Merges the groups containg elements p and q (if distinct)."""
        a = self.find(p)
        b = self.find(q)
        if a is not b: # only merge if different groups
            if a._size > b._size:
                b._parent = a
                a._size += b._size
                return a # I added to keep track of how many groups
            else:
                a._parent = b
                b._size += a._size
                return b # I added to keep track of how many groups

In [36]:
# Sollins Algorithm
# read in the graph
# file = open("mst5.txt","r") # not working, search for ugroup_index, vgroup_index 0 1
# file = open("mst9.txt","r")
# file = open("mst100_200.txt","r")
# file = open("mst100_4000.txt","r")
# file = open("mst500_2000.txt","r")
# file = open("mst500_4000.txt","r")
# file = open("mst500_6000.txt","r")
# file = open("mst500_8000.txt","r")
# file = open("mst500_10000.txt","r")
# file = open("mst500_12000.txt","r")
# file = open("mst1000_2000.txt","r")
# file = open("mst1000_4000.txt","r")
# file = open("mst1000_6000.txt","r")
# file = open("mst1000_8000.txt","r")
# file = open("mst1000_10000.txt","r")
# file = open("mst1000_12000.txt","r")

# file = open("mst2000_12000.txt","r")
# file = open("mst2000_14000.txt","r")
# file = open("mst2000_20000.txt","r")
# file = open("mst2000_30000.txt","r")
# file = open("mst2000_40000.txt","r")
file = open("mst2000_50000.txt","r")

#file = open("mst15.txt","r")
# file = open("distconected1.txt","r")
for i in range(2):
    file.readline()
infoline = file.readline()
file.readline() # extra line
paramline = file.readline()
linelist = paramline.split()
n_nodes = int(linelist[2])
n_arcs = int(linelist[3])

# setup the data structures
P = Partition() # a disjoint set
Grouplist = [] # a list to hold the Groups of the Partition
Vertexlist = [] # a non changing list of the pointers to the Position objects for each vertex
Group_to_Grouplistindex = {} # a dictionary with Group objects as keys and the indices as values
                                # the indices correspond to where the Group is in the Grouplist
Heaplist = [] # a list to hold the Heaps corresponding to each Group in the Grouplist

numGroups = n_nodes
for i in range(0,n_nodes):
    Grouplist.append(P.make_group(i))
    Vertexlist.append(Grouplist[i])
    Group_to_Grouplistindex[Grouplist[i]] = i
    Heaplist.append(None)

for  line in file: # loop through edges
    linelist = line.split()
    u = int(linelist[1])
    v = int(linelist[2])
    w = int(linelist[3])
    if u > v:
        u , v = v , u 
#     print(w,(u,v))
    H = Heap(w, (u,v))
    Heaplist[u] = Heap.meld(H,Heaplist[u])
    H = Heap(w, (u,v))
    Heaplist[v] = Heap.meld(H,Heaplist[v])

mst = []
totalloops = 0
disconectedGraph = False 
while numGroups > 1:
    if totalloops > n_arcs :
        print("###############infinite loop detected")
        disconectedGraph = True
        break
    totalloops = totalloops + 1
#     print("totalloops",totalloops)
    i = 0
    
    while i < numGroups:
        
#         print("The Groups when numGroups, i = ", numGroups,i)
        
#         print("There are the Heaps when numGroups, i = ", numGroups, i)
#         for j in range(0,numGroups):
#             print("*********Heap  ",j ,"  *************")
#             if Heaplist[j] == None:
#                 print("None")
#             else:
#                 Heaplist[j].printout()

#         for kk in range(numGroups):
#             index = Group_to_Grouplistindex[P.find(Grouplist[kk])]
#             if index != kk:
#                 print("index",index)
#                 for j in range(0,numGroups):
#                     print("*********Heap  ",j ,"  *************")
#                     if Heaplist[j] == None:
#                         print("None")
#                     else:
#                         Heaplist[j].printout()
#             print(Group_to_Grouplistindex[P.find(Grouplist[kk])])
        if Heaplist[i] == None:
            disconectedGraph = True
            break
        edge_weight , (u,v) = Heaplist[i]._key, Heaplist[i]._othervalues
        Heaplist[i] = Heaplist[i].deletemin()
        ugroup = P.find(Vertexlist[u])
        vgroup = P.find(Vertexlist[v])
        ugroup_index = Group_to_Grouplistindex[ugroup]
        vgroup_index = Group_to_Grouplistindex[vgroup]

        if ugroup_index == vgroup_index:
            pass # don't add this edge to the mst
        else:
            mst.append(((u,v),edge_weight))
            if ugroup_index < vgroup_index:
                Grouplist[ugroup_index] = P.union(ugroup,vgroup)
                Heaplist_ugroup_index = Heaplist[ugroup_index] # to avoid a swap in the pointers in Heaplist
                Heaplist_vgroup_index = Heaplist[vgroup_index]
                Heaplist[ugroup_index] = Heap.meld(Heaplist_ugroup_index, Heaplist_vgroup_index)
                Group_to_Grouplistindex[vgroup] = ugroup_index #update where this group is now
                if vgroup_index != numGroups -1:
                    Grouplist[vgroup_index], Heaplist[vgroup_index] = Grouplist[numGroups-1], Heaplist[numGroups-1]
                    Group_to_Grouplistindex[Grouplist[numGroups-1]] = vgroup_index # update where this group is now
                Grouplist[numGroups-1], Heaplist[numGroups-1] = None, None  # this line is needed, mst6.txt fails otherwis
                numGroups = numGroups - 1
            else: # ugroup_index > vgroup_index
                Grouplist[vgroup_index] = P.union(ugroup,vgroup)
                Heaplist_ugroup_index = Heaplist[ugroup_index]
                Heaplist_vgroup_index = Heaplist[vgroup_index]
                Heaplist[vgroup_index] = Heap.meld(Heaplist_vgroup_index,Heaplist_ugroup_index)
                Group_to_Grouplistindex[ugroup] = vgroup_index #update where this group is now
                if ugroup_index != numGroups -1:
                    Grouplist[ugroup_index], Heaplist[ugroup_index] = Grouplist[numGroups-1], Heaplist[numGroups-1]
                    Group_to_Grouplistindex[Grouplist[numGroups-1]] = ugroup_index # update where this group is now
                Grouplist[numGroups-1], Heaplist[numGroups-1] = None, None
                numGroups = numGroups - 1
        i = i + 1
        

if disconectedGraph:
    print("The graph is disconnected, the output is a spanning forest")
    
# print("number of arcs in mst", len(mst)) 
n_mstarcs = len(mst)
print("DIMACS Output\n")
print("c Minimum Spanning Tree")
print("c")
infolinelist = infoline.split(';')
infolinelist[1] = "arcs:  " + str(n_mstarcs)
# print(infolinelist)
infolinenew = infolinelist[0] + "; " + infolinelist[1] + " ;" + infolinelist[2]
print(infolinenew, end = "")
print("c")
print(paramline, end="")
for edge in mst:
#     print(edge)
    print("e", edge[0][0], edge[0][1], edge[1])  

DIMACS Output

c Minimum Spanning Tree
c
c nodes:  2000 ; arcs:  1999 ; cost range:  -5 - 10
c
p mst  2000 50000
e 0 1347 -5
e 1 333 -5
e 2 1253 -5
e 3 1436 -5
e 4 1406 -5
e 5 576 -5
e 6 1031 -5
e 7 1589 -5
e 8 344 -5
e 9 671 -5
e 10 1424 -5
e 11 701 -5
e 12 510 -5
e 13 689 -5
e 14 116 -5
e 15 1841 -5
e 16 683 -5
e 17 1041 -5
e 18 1295 -5
e 19 1294 -5
e 20 1515 -5
e 21 327 -5
e 22 1365 -5
e 23 1704 -5
e 24 1861 -5
e 25 1632 -5
e 26 72 -5
e 27 231 -5
e 28 463 -5
e 29 427 -5
e 30 953 -5
e 31 772 -5
e 32 1269 -4
e 33 1637 -5
e 34 394 -5
e 35 1177 -5
e 36 1576 -5
e 37 260 -5
e 38 319 -5
e 39 1064 -5
e 40 1176 -5
e 41 1433 -5
e 42 894 -5
e 43 1651 -5
e 44 1512 -5
e 45 1043 -5
e 46 1087 -5
e 47 1361 -5
e 48 1468 -5
e 49 830 -5
e 50 1490 -5
e 51 140 -5
e 52 1889 -5
e 53 1098 -5
e 54 1269 -5
e 55 1072 -5
e 56 1376 -5
e 57 892 -5
e 58 707 -5
e 59 1236 -4
e 60 374 -5
e 61 867 -5
e 62 1187 -5
e 63 78 -5
e 64 961 -5
e 65 290 -5
e 66 1898 -5
e 67 801 -5
e 68 1665 -5
e 69 379 -4
e 70 407 -5
e 71 194

e 111 1273 -5
e 923 1859 -5
e 114 806 -5
e 839 1985 -5
e 117 733 -5
e 118 1001 -5
e 119 122 -5
e 120 1474 -5
e 1678 1878 -5
e 891 911 -5
e 123 209 -5
e 102 124 -5
e 125 1185 -5
e 126 283 -5
e 1133 1791 -5
e 129 774 -5
e 546 1481 -5
e 131 314 -5
e 212 1013 -5
e 1037 1207 -5
e 1008 1064 -5
e 584 1888 -5
e 137 1504 -5
e 1074 1129 -5
e 139 1573 -5
e 141 164 -5
e 37 142 -5
e 1565 1856 -5
e 145 1185 -5
e 42 1853 -5
e 147 1246 -5
e 149 374 -5
e 150 986 -5
e 151 780 -5
e 152 1477 -5
e 153 1938 -5
e 1417 1944 -5
e 1416 1725 -5
e 158 851 -5
e 159 766 -5
e 160 1860 -5
e 1327 1910 -5
e 458 1663 -5
e 165 1985 -5
e 1407 1833 -5
e 41 168 -5
e 169 1179 -5
e 699 1081 -5
e 620 1011 -5
e 877 1061 -4
e 557 654 -5
e 1824 1970 -5
e 176 1724 -5
e 178 639 -5
e 190 1820 -5
e 1265 1813 -5
e 182 205 -5
e 1332 1373 -5
e 1324 1678 -5
e 836 1814 -5
e 187 688 -5
e 41 188 -5
e 1645 1718 -5
e 1115 1151 -5
e 191 1552 -5
e 1050 1387 -5
e 1659 1787 -5
e 194 839 -5
e 195 537 -5
e 1078 1192 -5
e 1740 1801 -4
e 1489 1717 -5

e 693 1222 -5
e 577 1223 -5
e 504 590 -5
e 631 1225 -5
e 340 424 -5
e 1227 1563 -5
e 373 1759 -5
e 161 665 -5
e 1201 1551 -5
e 1792 1976 -5
e 163 1301 -4
e 359 1647 -5
e 1062 1246 -5
e 1254 1701 -5
e 688 1700 -5
e 983 1940 -4
e 680 1349 -5
e 759 1451 -5
e 801 1660 -5
e 3 1390 -5
e 561 576 -5
e 1517 1811 -5
e 11 1455 -5
e 510 1826 -5
e 683 1352 -5
e 583 1439 -5
e 18 1808 -5
e 907 1432 -5
e 327 609 -5
e 1145 1365 -5
e 1575 1848 -5
e 572 1696 -5
e 54 1798 -5
e 923 1582 -5
e 438 1294 -5
e 37 846 -5
e 300 616 -5
e 1188 1687 -5
e 324 1544 -5
e 904 1651 -5
e 1025 1426 -5
e 46 263 -5
e 315 1971 -5
e 16 49 -5
e 1282 1747 -5
e 140 1249 -5
e 1422 1809 -5
e 1098 1206 -5
e 1945 1974 -5
e 1496 1897 -5
e 1056 1079 -5
e 1236 1673 -5
e 374 1402 -5
e 362 659 -5
e 1047 1420 -5
e 290 1498 -5
e 324 391 -5
e 96 956 -5
e 739 1412 -5
e 702 761 -5
e 1101 1413 -5
e 404 454 -5
e 560 1647 -5
e 108 1654 -5
e 1102 1269 -5
e 738 1403 -4
e 126 828 -5
e 1401 1745 -5
e 926 1400 -5
e 1483 1825 -5
e 257 1051 -5
e 1156 13